In [2]:
import os
import re
import json
import subprocess

from openai import OpenAI
import sys
sys.path.append('../')
from prompts.design_prompt import design_prompt
from prompts.parsing_prompt import parsing_prompt
from prompts.scripting_prompt import scripting_prompt

In [3]:
with open('../config/config.json') as f:
     config = json.load(f)

In [4]:
OPENSCAD_EXEC = config["OPENSCAD_EXEC_PATH"]
FILE_DIR = "../common/herringbon"
DESIGN_FILENAME = "gears_design.md"
PARAMETERS_FILENAME = "gear_parameters.json"
SCRIPT_FILENAME = "gears_script.scad"
if not os.path.exists(FILE_DIR):
    os.makedirs(FILE_DIR)

In [26]:
client = OpenAI(api_key=config['OPENAI_API_KEY'])

In [14]:
design_description = f""" 
Suppose there is a one-stage system with two herringbone gears with a 20 degree pitch angle.
The height of both gears are 25 and 16 degree respectively, and the pressure angle are both 20 degree.
Currently, there is a requirements for the reduction system that the desired gear ratio is 2:1.
Set the driving gear to the center of the plane, which is the coordinate (0, 0).
Based on the given information, computer module number, teeth number of both gears and the coordniate of the driven gear centers.
"""

In [15]:
design_messages=[
        {"role": "system", "content": design_prompt},
        {"role": "user", "content": design_description},
    ]

In [16]:
# stop_tokens = ['']
# collected_tokens = []

# res = client.chat.completions.create(
#     model=config['GPT_MODEL'],
#     messages=design_messages,
#     stream=True,
# )

# for chunk in res:
#     temp_tokens = chunk.choices[0].delta.content
#     if temp_tokens not in stop_tokens:
#         collected_tokens.append(temp_tokens)
#     else:
#         chunk_res = "".join(collected_tokens).strip()
#         print(chunk_res)
#         collected_tokens = []

res = client.chat.completions.create(
    model=config['GPT_MODEL'],
    messages=design_messages,
    temperature=0
)

In [17]:
print(res.choices[0].message.content)

To design a gear system with the given requirements, we need to determine the module, number of teeth, and the coordinates of the driven gear center. Let's start by defining the gear ratio and the basic gear equation.

### Step 1: Gear Ratio
The gear ratio (i) is given as 2:1, which means the driven gear must have twice as many teeth as the driving gear.

### Step 2: Basic Gear Equation
The basic gear equation relates the module (m), number of teeth (Z), and pitch diameter (d) as follows:

\[ d = m \cdot Z \]

### Step 3: Center Distance
The center distance (a) between the two gears is given as 60mm. For a gear ratio of 2:1, the pitch diameters of the gears must satisfy the following relationship:

\[ a = \frac{d_{driving} + d_{driven}}{2} \]

Given that the gear ratio is 2:1, we can express the pitch diameters in terms of the module and number of teeth:

\[ a = \frac{m \cdot Z_{driving} + m \cdot 2 \cdot Z_{driving}}{2} \]
\[ a = \frac{m \cdot Z_{driving} (1 + 2)}{2} \]
\[ a = \frac{3

In [26]:
with open(os.path.join(FILE_DIR, DESIGN_FILENAME), "w") as f:
    f.write(res.choices[0].message.content)

In [27]:
# Mistaken on the distance from the obtained result, which should be (60, 0) for the driven gear

parsing_description = \
f"gear 1 (driving gear): herringbone gear, module 2mm, 20 teeth, pitch diameter 40mm, height 25mm, pressure angle 20 degrees, coordinate (0,0) " + \
f"pitch angle 20 degrees \n" + \
f"gear 2 (driven gear): herringbone gear, module 2mm, 40 teeth, pitch diameter 80mm, height 16mm, pressure angle 20 degrees, coordinate (60,0) " + \
f"pitch angle 20 degrees"

In [28]:
parsing_messages = [
    {"role": "system", "content": parsing_prompt},
    {"role": "user", "content": parsing_description},
]

In [29]:
parsing_response = client.chat.completions.create(
    model=config['GPT_MODEL'],
    response_format={ "type": "json_object" },
    messages=parsing_messages,
)

In [30]:
gears_parameters = parsing_response.choices[0].message.content.strip()
print(gears_parameters)

{
  "gear 1": {
    "gear_type": "herringbone",
    "source": "driving",
    "coordinate": {
      "x": 0,
      "y": 0,
      "z": 0,
      "alpha": 0,
      "beta": 0,
      "gamma": 0
    },
    "unit": "mm",
    "module": 2,
    "teeth": 20,
    "height": 25,
    "pitch_d": 40,
    "pitch_angle": 20,
    "pressure_angle": 20
  },
  "gear 2": {
    "gear_type": "herringbone",
    "source": "driven",
    "coordinate": {
      "x": 60,
      "y": 0,
      "z": 0,
      "alpha": 0,
      "beta": 0,
      "gamma": 0
    },
    "unit": "mm",
    "module": 2,
    "teeth": 40,
    "height": 16,
    "pitch_d": 80,
    "pitch_angle": 20,
    "pressure_angle": 20
  }
}


In [33]:
with open(os.path.join(FILE_DIR, PARAMETERS_FILENAME), "w") as f:
    f.write(gears_parameters)

In [34]:
with open(os.path.join(FILE_DIR, PARAMETERS_FILENAME), "r") as f:
    parameters = json.load(f)
parameters = str(parameters)

scripting_description = f"Write a script according to provided parameters of gears \n" + parameters

In [35]:
script_messages = [
    {"role": "system", "content": scripting_prompt},
    {"role": "user", "content": scripting_description}
]

In [36]:
print(scripting_prompt)


You are an expert in 3D modeling with Computer-aided design package.
You need to write a model script in OpenSCAD to parametrically build the 3D geometry with parameters in the given json file. 
The json format is given by: {'gear 1': {'gear_type': '', 'source': '', 'coordinate': {'x': '', 'y': '', 'z': '', 'alpha': '', 'beta': '', 'gamma': ''}, 'unit': '', 'module': '', 'teeth': '', 'height': '', 'pitch_d': '', 'pitch_angle': '', 'pressure_angle': ''}}

The process of writing script is described as follow:

Firstly, you should set the spatial position of items based on 'coordinate' parameters.
If 'gear_type' are not 'spur', the pitch_angle should be inverse to each other.
If the 'source' are not 'driving', the 'gamma' is equal to 360 / (teeth * 2), then write to the script: 
translate([x,y,z]) rotate([alpha, beta, gamma])

Next, based on the 'gear_type' in the Json file to choose the most related command and then write to the script from following functions:
spur gear => gear(m=modul

In [37]:
scripting_response = client.chat.completions.create(
    model=config['GPT_MODEL'],
    messages=script_messages,
    # temperature=0
)

In [38]:
def script_postprocessing(script):
    # To remove textual perfix or suffix information in the raw script generated by GPT-4
    pattern = re.compile(r'```(?:openscad|scad)(.*?)```', re.DOTALL)
    matches = re.findall(pattern, script)
    clean_script = ""
    if len(matches) > 0:
        for m in matches:
            clean_script = clean_script + m.strip()
    else:
        clean_script = script
    return clean_script

In [39]:
script = scripting_response.choices[0].message.content.strip()
script = script_postprocessing(script)
print(script)

translate([0,0,0]) rotate([0,0,0])
gear_herringbone(m=2, z=20, h=25, w=20, w_helix=20);
translate([60,0,0]) rotate([0,0,360/(40*2)])
gear_herringbone(m=2, z=40, h=16, w=20, w_helix=-20);


In [40]:
with open(os.path.join(FILE_DIR, SCRIPT_FILENAME), "w") as f:
    f.write("include <gears.scad>")

with open(os.path.join(FILE_DIR, SCRIPT_FILENAME), "a") as f:
    f.write("\n" + script)

In [5]:
subprocess.Popen([OPENSCAD_EXEC, os.path.join(FILE_DIR, SCRIPT_FILENAME)])

<Popen: returncode: None args: ['D:\\openSCAD\\install\\openscad.exe', '../c...>